In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math
from numpy.linalg import inv
import numpy.matlib

In [ ]:
# Python adaptation of genlsq.m written by Carl Tape
# Applied Seismology, GEOS 626, University of Alaska Fairbanks
# Python coding by Nealey Sims
#
# genlsq = Generalized Least Squares
# Template script for the iterative quasi-Newton method for a 4-parameter
# inversion for epicenter (xs, ys), origin time (ts), and velocity (V).
# The algorithm employs generalized least squares, where by both data
# covariances and model covariances are used.
#
# Background reading: Tarantola book (2005), Ch. 3 and Appendix 6.22
#
# calls forward_epicenter.ipynb, plot_covsamples
#
# Carl Tape, 2012-01-01
#

In [ ]:
#=========================================
# USER INPUT

nsamples = 1000;
irandom_initial_model = 0;      # 0(fixed), 1(random)
irandom_target_model = 0;       # 0(fixed), 1(random)
idata_errors = 2;               # 0(none),  1(random), 2(fixed)
ifig = 1;                       # 0,1

#=========================================

inormalization = 1;
stnsamples = str(nsamples) + ' samples'
stlabS = ('Sd(m^k)','Sm(m^k)','S(m^k) = Sd + Sm')

In [ ]:
#---------------------------------------------
# FORWARD PROBLEM    
%run ./forward_epicenter.ipynb      # KEY COMMAND

In [ ]:
def plot_histo(hdat,edges,itype=2,make_plot=True):
    #PLOT_HISTO plot a histogram with cyan bars and black boundaries
    #
    # INPUT
    #   hdat        input data to bin
    #   edges       vector defining the edges of the bins (for hdat)
    #   itype       optional: type of histogram (=1,2,3) [default = 2]
    #   make_plot   optional: plot histogram [default = true]
    #hdat = hdat.flatten();
    #barcolor = [1, 1, 1]*0.8;
    
    # bin width (only relevant if bins are the same width)
    dbin = edges[1] - edges[0]
    hedges=np.append(edges,edges[-1]+5)
    Ntotal = len(hdat);
    N,b = np.histogram(hdat,hedges);
    if itype ==1:
        Nplot = N; xlab = 'Count'
    if itype ==2: 
        Nplot = np.divide(N,Ntotal); xlab = 'Fraction'
    if itype ==3: 
        Nplot = np.divide(np.divide(N,Ntotal),dbin); xlab = 'PDF'
        #if len(unique(edges)) > 1:
        if np.std(np.diff(edges))/np.mean(np.diff(edges)) > 1e-4:       # ad hoc criterion
            print(np.unique(np.diff(edges)))
            print('PDF is not implemented to allow bins with varying widths')
            
    elif itype!=1 and itype!=2 and itype!=3: 
        print('itype = %i -- it must be 1,2, or 3'%(itype)) 

    if make_plot==True:
        plt.bar(edges,Nplot, width=0.8*dbin);
        plt.xlim([min(edges), max(edges)]);
        plt.ylabel('%s (N=%i)'% (xlab,Ntotal))
        
        if len(hdat) != np.sum(N):
            print('(plot_histo.m): You may want to extend the histogram edges -->');
            print(' there are %i/%i input that are outside the specified range'%
                (len(hdat)-np.sum(N),len(hdat)))
            #disp(sprintf(' the number of input (%i) does not equal the sum of bin counts (%i).',length(hdat),sum(N)));
    
    

In [ ]:
# predictions for prior and initial models (not necessary)
dprior   = d(mprior)
dinitial = d(minitial);

if ifig==1:
    # plot different histograms of properties of the prior model covariance samples
    ##figure; nr=2; nc=2;
    fig=plt.figure(figsize=(9,9))
    for kk in range(nparm):
        sigma = sigma_prior[kk]
        #edges = [-4*sigma: sigma/2 : 4*sigma]
        edges=np.arange(-4*sigma,4*sigma,sigma/2)
        etemp = cov_samples_m[kk,:]
        plt.subplot(2,2,kk+1)
        plt.grid()
        plot_histo(etemp,edges,2);
        #plt.bar(etemp)
        plt.ylim([0, 0.4])
        plt.title('mprior samples: Model parameter '+str(kk)+' ('+str(mlabs[kk])+')\n'+'mean = %.5f; std = %.5f' % (np.mean(etemp),np.std(etemp)))
        plt.tight_layout()
    # plot different histograms of properties of the data covariance samples
    ##figure; nr=4; nc=3;
    fig2 = plt.figure(figsize=(9,11))
    for ii in range(ndata):
        sigma = sigma_obs[ii];
        edges=np.arange(-4*sigma,4*sigma,sigma/2)
        etemp = cov_samples_d[ii,:]
        plt.subplot(4,3,ii+1); 
        plot_histo(etemp,edges); 
        plt.ylim([0, 0.4]);
        plt.title('Data index ' + str(ii) +'\n'+'mean = %.5f; std = %.5f' % (np.mean(etemp),np.std(etemp))) 
        plt.tight_layout()
    fig3 = plt.figure(figsize=(10,10))
    plt.plot(dobs_samples,'.-');
    p1 = plt.plot(dprior,'bo-',linewidth=2,markersize=10,markerfacecolor='b',markeredgecolor='w')
    p2 = plt.plot(dinitial,'ko-',linewidth=2,markersize=10,markerfacecolor='k',markeredgecolor='w');
    p3 = plt.plot(dtarget,'ro--',linewidth=2,markersize=10,markerfacecolor='r',markeredgecolor='w');
    p4 = plt.plot(dobs,'ro-',linewidth=2,markersize=10,markerfacecolor='r',markeredgecolor='w');
    plt.legend([p1[0], p2[0], p3[0], p4[0]],['g(mprior)','g(minitial)','g(mtarget)','g(mtarget) + errors',
        'location','northwest'])
    #title(' BLACK = d(mprior);  RED DASHED = d(mtarget);  RED = d(mtarget) + errors');
    plt.xlim([0.5, ndata+0.5]); #set(gca,'xtick',[1:ndata]);
    plt.xlabel('Data index'); plt.ylabel('Prediction value, g(m)');
    


In [ ]:
#---------------------------------------------
# MISFIT FUNCTION : least squares, Tarantola (2005), Eq. 6.251
# (This calls the function d to compute the predictions.)

# data misfit
def Sd(m,dobs,icobs):
    sd=np.dot(np.dot(np.dot(0.5,np.transpose(np.transpose([d(m)])-dobs)),icobs),(np.transpose([d(m)])-dobs))
    return sd
# model misfit (related to regularization)
def Sm(m,mprior,icprior):
    sm=np.dot(np.dot(np.dot(0.5,np.transpose(np.transpose([m])-mprior)),icprior),(np.transpose([m]) - mprior))
    return sm
# total misfit
def S(m,dobs,mprior,icobs,icprior):
    s=Sd(m,dobs,icobs) + Sm(m,mprior,icprior)
    return s

# initial model
#mnew = mprior;     # prior model
mnew = minitial;
Sd_0 = Sd(mnew,dobs,icobs);
Sm_0 = Sm(mnew,mprior,icprior);
S_0  = S(mnew,dobs,mprior,icobs,icprior);
stS0 = ' S(m0) = %.3f = %.3f(D) + %.3f(M)'% (S_0,Sd_0,Sm_0)
print(stS0);

niter = input(' Select the number of iterations (< 10) or 0 to exit: ');
if niter==0:
    quit()

In [ ]:
if niter !=0:
    # initialize arrays
    niter=int(niter)
    iter_vec = np.transpose(np.arange(0,niter))
    Sd_vec = np.zeros((niter+1,1));
    Sm_vec = np.zeros((niter+1,1));
    S_vec = np.zeros((niter+1,1));
    
    # misfit for initial model
    Sd_vec[0] = Sd_0;
    Sm_vec[0] = Sm_0;
    S_vec[0] = S_0;
    
    for nn in range(niter):
        #///////////////////////////////
        # CODE HERE for quasi-Newton algorithm
        
        
        
        # fill misfit function S_vec, Sd_vec, Sm_vec for plotting later
        #Sd_vec(nn+1) = 
        #Sm_vec(nn+1) = 
        #S_vec(nn+1) = 
        
        #///////////////////////////////